In [1]:
import random
import numpy as np
import json
from tqdm import tqdm
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.spatial.distance import pdist, squareform
from scipy.optimize import linear_sum_assignment
from collections import Counter
import time
import os
import pandas as pd
from scipy.signal import correlate
np. set_printoptions(threshold=np. inf)

In [2]:
date = 'Oct4th-'
filename = ''.join((date,'scan_stats.json'))
filename

'Oct4th-scan_stats.json'

In [3]:
import scan
import simulate_data

In [4]:
default_params = {
    'M':1,
    'N':40,
    'D':51,
    'T':1000,
    'seed':0,
    'num_SM_events':16,
    'SM_total_spikes':10,
    'noise':100
}
scan_dict = {
    'M':[1,4,8,16,32],
    'N':[10,20,40,80,120],
    'D':[11,31,51,71,101],
    'num_SM_events':[2,4,8,16,32],
    'SM_total_spikes':[3,5,10,20,50],
    'noise':[0,50,100,500,1000]
}

In [5]:
def get_acc(matrix_x,matrix_y):
    # Calculate cross-correlation matrix
    cc = np.zeros((matrix_x.shape[2], matrix_y.shape[2]))

    for x_channel_idx in range(matrix_x.shape[2]):
        for y_channel_idx in range(matrix_y.shape[2]):
            cc[x_channel_idx, y_channel_idx], _ = max_overlap(matrix_x[...,x_channel_idx],matrix_y[...,y_channel_idx])
    SM_acc = np.max(cc,axis=1)
    return SM_acc, cc

In [6]:
def max_overlap(image, kernel):
    result = np.zeros((image.shape[1]+kernel.shape[1]-1))
    for n in range(image.shape[0]):
        result += correlate(image[n,:], kernel[n,:], mode = 'full')
    return np.max(result)/max(np.sum(image),np.sum(kernel)), np.argmax(result)

In [7]:
def get_imgs(K_dense, pattern_template):
    if len(pattern_template) == 0:
        print('FAIL')
        return pattern_template, None
    
    win_size = (K_dense.shape[0],1+max([max(k[:,0]) for k in pattern_template]))
    pattern_img = np.zeros((len(pattern_template),*win_size))
    for p,pattern in enumerate(pattern_template):
        for (i,j) in pattern:
            pattern_img[p,j,i] = 1
            
    pattern_img = pattern_img.transpose((1,2,0))
    
    return pattern_template, pattern_img

In [8]:
from scipy.signal import correlate
def get_acc(ground_truths,detected_patterns):
    # Calculate cross-correlation matrix
    cross_corr_matrix = np.zeros((ground_truths.shape[2], detected_patterns.shape[2]))
    SM_acc = np.zeros((ground_truths.shape[2]))
    
    if len(detected_patterns) == 0:
        return SM_acc, cross_corr_matrix
    
    for ground_truths_idx in range(ground_truths.shape[2]):
        for detected_patterns_idx in range(detected_patterns.shape[2]):
            cross_corr = np.zeros((ground_truths.shape[1]+detected_patterns.shape[1]-1))
            for n in range(ground_truths.shape[0]):
                cross_corr += correlate(ground_truths[n, :, ground_truths_idx], detected_patterns[n, :, detected_patterns_idx], mode='full')
            max_corr = np.max(cross_corr) / max(np.sum(ground_truths[...,ground_truths_idx]),np.sum(detected_patterns[...,detected_patterns_idx]))
            cross_corr_matrix[ground_truths_idx, detected_patterns_idx] = max_corr
#     print(cross_corr_matrix)
#     print( np.sum(ground_truths[...,ground_truths_idx]))
    SM_acc = np.max(cross_corr_matrix,axis=1)
    return SM_acc, cross_corr_matrix

In [9]:
def check_ground_truth(pattern_template, K_dense):
    _, pattern_img = get_imgs(K_dense, pattern_template)
    SM_acc, cc = get_acc(K_dense, pattern_img)
    return SM_acc, cc, pattern_img
    

In [10]:
param_combinations = []
for param_name, param_values in scan_dict.items():
    for param_value in param_values:
        params = default_params.copy()
        params[param_name] = param_value
        param_combinations.append(params)

In [11]:
len(param_combinations)

30

In [16]:
K_dense.shape

(40, 51, 8)

In [ ]:
import numpy as np
import random
import os
import pandas as pd
import json
from tqdm import tqdm

# Define default parameters and scan values
default_params = {
    'M': 8,
    'N': 40,
    'D': 51,
    'T': 1000,
    'seed': 0,
    'num_SM_events': 8,
    'SM_total_spikes': 10,
    'noise': 100
}

scan_dict = {
    'M': [1, 4, 8, 16, 32],
    'N': [10, 20, 40, 80, 120],
    'D': [11, 31, 51, 71, 101], 
    'num_SM_events': [2, 4, 8, 16, 32],
    'SM_total_spikes': [3, 5, 10, 20, 50],
    'noise': [0, 50, 100, 500, 1000]
}

# Generate all parameter combinations
param_combinations = []
for param_name, param_values in scan_dict.items():
    for param_value in param_values:
        params = default_params.copy()
        params[param_name] = param_value
        param_combinations.append(params)

num_samples = len(param_combinations)
trials = 100
results = []

# Iterate through parameter combinations
for idx, params in tqdm(enumerate(param_combinations)):
    for trial in range(trials):

        if os.path.isfile(filename):
            df = pd.read_json(filename)
            with open(filename, 'r') as results_file:
                results = json.load(results_file)
        params['seed'] = trial
        print(params)
        A_dense, A_sparse, B_dense, B_sparse, K_dense, K_sparse = simulate_data.generate_synthetic_data(params)
        start = time.time()
        pattern_template, sublist_keys_filt, window_time, cluster_time, sequence_time = scan.scan_raster(A_sparse[1], A_sparse[0], window_dim=params['D'])
        if len(pattern_template) != 0:
            SM_acc, cc, _ = check_ground_truth(pattern_template, K_dense)
        else:
            SM_acc, cc = np.zeros(K_dense.shape[2]), []
        end = time.time()
        result = {
            'M':params['M'],
            'N':params['N'],
            'D':params['D'],
            'T':params['T'],
            'num_SM_events':params['num_SM_events'],
            'SM_total_spikes':params['SM_total_spikes'],
            'noise':params['noise'],
            'window_time': window_time,
            'cluster_time': cluster_time,
            'sequence_time': sequence_time,
            'total_time': end-start,
            'total_spikes':len(A_sparse[1]),
            'total_patterns':len(pattern_template),
            'SM_acc':SM_acc.tolist()
        }

        results.append(result)

        with open(filename, 'w') as results_file:
            json.dump(results, results_file, indent=4)


0it [00:00, ?it/s]

{'M': 1, 'N': 40, 'D': 51, 'T': 1000, 'seed': 0, 'num_SM_events': 8, 'SM_total_spikes': 10, 'noise': 100}
180 Windows
5 patterns found...s... 80% 10.01 | opt_cutoff - 1.09 | most_detections - 7etections - 7
{'M': 1, 'N': 40, 'D': 51, 'T': 1000, 'seed': 1, 'num_SM_events': 8, 'SM_total_spikes': 10, 'noise': 100}
180 Windows
4 patterns found...s... 75% 10.01 | opt_cutoff - 0.61 | most_detections - 7etections - 7t_detections - 1
{'M': 1, 'N': 40, 'D': 51, 'T': 1000, 'seed': 2, 'num_SM_events': 8, 'SM_total_spikes': 10, 'noise': 100}
179 Windows
1 patterns found...s... 0%- 10.01 | opt_cutoff - 1.37 | most_detections - 7etections - 7t_detections - 2
{'M': 1, 'N': 40, 'D': 51, 'T': 1000, 'seed': 3, 'num_SM_events': 8, 'SM_total_spikes': 10, 'noise': 100}
180 Windows
1 patterns found...s... 0%- 10.01 | opt_cutoff - 1.09 | most_detections - 7etections - 7
{'M': 1, 'N': 40, 'D': 51, 'T': 1000, 'seed': 4, 'num_SM_events': 8, 'SM_total_spikes': 10, 'noise': 100}
180 Windows
5 patterns found...s..

1 patterns found...s... 0%- 10.01 | opt_cutoff - 1.09 | most_detections - 7etections - 7t_detections - 1
{'M': 1, 'N': 40, 'D': 51, 'T': 1000, 'seed': 76, 'num_SM_events': 8, 'SM_total_spikes': 10, 'noise': 100}
180 Windows
1 patterns found...s... 0%- 10.01 | opt_cutoff - 1.09 | most_detections - 7etections - 7
{'M': 1, 'N': 40, 'D': 51, 'T': 1000, 'seed': 77, 'num_SM_events': 8, 'SM_total_spikes': 10, 'noise': 100}
180 Windows
1 patterns found...s... 0%- 10.01 | opt_cutoff - 1.09 | most_detections - 7etections - 72_detections - 2
{'M': 1, 'N': 40, 'D': 51, 'T': 1000, 'seed': 78, 'num_SM_events': 8, 'SM_total_spikes': 10, 'noise': 100}
180 Windows
2 patterns found...s... 50% 10.01 | opt_cutoff - 1.37 | most_detections - 7etections - 7
{'M': 1, 'N': 40, 'D': 51, 'T': 1000, 'seed': 79, 'num_SM_events': 8, 'SM_total_spikes': 10, 'noise': 100}
180 Windows
2 patterns found...s... 50% 10.01 | opt_cutoff - 0.78 | most_detections - 7etections - 71
{'M': 1, 'N': 40, 'D': 51, 'T': 1000, 'seed': 

1it [00:52, 52.66s/it]

2 patterns found...s... 50% 10.01 | opt_cutoff - 0.78 | most_detections - 7etections - 7
{'M': 4, 'N': 40, 'D': 51, 'T': 1000, 'seed': 0, 'num_SM_events': 8, 'SM_total_spikes': 10, 'noise': 100}
417 Windows
12 patterns found...... 92% 10.01 | opt_cutoff - 0.51 | most_detections - 7etections - 75
{'M': 4, 'N': 40, 'D': 51, 'T': 1000, 'seed': 1, 'num_SM_events': 8, 'SM_total_spikes': 10, 'noise': 100}
418 Windows
14 patterns found...... 93% 10.01 | opt_cutoff - 0.51 | most_detections - 7etections - 7t_detections - 6
{'M': 4, 'N': 40, 'D': 51, 'T': 1000, 'seed': 2, 'num_SM_events': 8, 'SM_total_spikes': 10, 'noise': 100}
419 Windows
15 patterns found...... 93% 10.01 | opt_cutoff - 0.51 | most_detections - 7etections - 7t_detections - 6
{'M': 4, 'N': 40, 'D': 51, 'T': 1000, 'seed': 3, 'num_SM_events': 8, 'SM_total_spikes': 10, 'noise': 100}
418 Windows
15 patterns found...... 93% 10.01 | opt_cutoff - 0.47000000000000003 | most_detections - 7etections - 77
{'M': 4, 'N': 40, 'D': 51, 'T': 10

...

{'M': 8, 'N': 40, 'D': 51, 'T': 1000, 'seed': 0, 'num_SM_events': 16, 'SM_total_spikes': 10, 'noise': 100}
1353 Windows
11 patterns found...... 91% 10.01 | opt_cutoff - 0.38 | most_detections - 15tections - 155
{'M': 8, 'N': 40, 'D': 51, 'T': 1000, 'seed': 1, 'num_SM_events': 16, 'SM_total_spikes': 10, 'noise': 100}
1355 Windows
21 patterns found...... 95% 10.01 | opt_cutoff - 0.42 | most_detections - 15tections - 155detections - 1
{'M': 8, 'N': 40, 'D': 51, 'T': 1000, 'seed': 2, 'num_SM_events': 16, 'SM_total_spikes': 10, 'noise': 100}
1354 Windows
5 patterns found...s... 80% 10.01 | opt_cutoff - 0.47000000000000003 | most_detections - 15tections - 155
{'M': 8, 'N': 40, 'D': 51, 'T': 1000, 'seed': 3, 'num_SM_events': 16, 'SM_total_spikes': 10, 'noise': 100}
1353 Windows
14 patterns found...... 93% 10.01 | opt_cutoff - 0.42 | most_detections - 15tections - 155
{'M': 8, 'N': 40, 'D': 51, 'T': 1000, 'seed': 4, 'num_SM_events': 16, 'SM_total_spikes': 10, 'noise': 100}
1346 Windows
8 patte

12 patterns found...... 92% 10.01 | opt_cutoff - 0.47000000000000003 | most_detections - 15tections - 155
{'M': 8, 'N': 40, 'D': 51, 'T': 1000, 'seed': 39, 'num_SM_events': 16, 'SM_total_spikes': 10, 'noise': 100}
1360 Windows
20 patterns found...... 95% 10.01 | opt_cutoff - 0.42 | most_detections - 15tections - 155
{'M': 8, 'N': 40, 'D': 51, 'T': 1000, 'seed': 40, 'num_SM_events': 16, 'SM_total_spikes': 10, 'noise': 100}
1357 Windows
19 patterns found...... 95% 10.01 | opt_cutoff - 0.38 | most_detections - 15tections - 155
{'M': 8, 'N': 40, 'D': 51, 'T': 1000, 'seed': 41, 'num_SM_events': 16, 'SM_total_spikes': 10, 'noise': 100}
1364 Windows
15 patterns found...... 93% 10.01 | opt_cutoff - 0.38 | most_detections - 15tections - 155
{'M': 8, 'N': 40, 'D': 51, 'T': 1000, 'seed': 42, 'num_SM_events': 16, 'SM_total_spikes': 10, 'noise': 100}
1357 Windows
8 patterns found...s... 88% 10.01 | opt_cutoff - 0.47000000000000003 | most_detections - 15tections - 155
{'M': 8, 'N': 40, 'D': 51, 'T':

14 patterns found...... 93% 10.01 | opt_cutoff - 0.42 | most_detections - 15tections - 155
{'M': 8, 'N': 40, 'D': 51, 'T': 1000, 'seed': 77, 'num_SM_events': 16, 'SM_total_spikes': 10, 'noise': 100}
1361 Windows
17 patterns found...... 94% 10.01 | opt_cutoff - 0.51 | most_detections - 15tections - 154
{'M': 8, 'N': 40, 'D': 51, 'T': 1000, 'seed': 78, 'num_SM_events': 16, 'SM_total_spikes': 10, 'noise': 100}
1361 Windows
22 patterns found...... 95% 10.01 | opt_cutoff - 0.38 | most_detections - 15tections - 155
{'M': 8, 'N': 40, 'D': 51, 'T': 1000, 'seed': 79, 'num_SM_events': 16, 'SM_total_spikes': 10, 'noise': 100}
1352 Windows
5 patterns found...s... 80% 10.01 | opt_cutoff - 0.42 | most_detections - 15tections - 155
{'M': 8, 'N': 40, 'D': 51, 'T': 1000, 'seed': 80, 'num_SM_events': 16, 'SM_total_spikes': 10, 'noise': 100}
1360 Windows
11 patterns found...... 91% 10.01 | opt_cutoff - 0.42 | most_detections - 15tections - 155
{'M': 8, 'N': 40, 'D': 51, 'T': 1000, 'seed': 81, 'num_SM_eve

19it [1:23:03, 273.83s/it]

{'M': 8, 'N': 40, 'D': 51, 'T': 1000, 'seed': 0, 'num_SM_events': 32, 'SM_total_spikes': 10, 'noise': 100}
2567 Windows
12 patterns found...... 92% 10.01 | opt_cutoff - 0.56 | most_detections - 30tections - 30
{'M': 8, 'N': 40, 'D': 51, 'T': 1000, 'seed': 1, 'num_SM_events': 32, 'SM_total_spikes': 10, 'noise': 100}
2578 Windows
17 patterns found...... 94% 10.01 | opt_cutoff - 0.47000000000000003 | most_detections - 18tections - 188
{'M': 8, 'N': 40, 'D': 51, 'T': 1000, 'seed': 2, 'num_SM_events': 32, 'SM_total_spikes': 10, 'noise': 100}
2565 Windows
1 patterns found...s... 0%- 10.01 | opt_cutoff - 0.67 | most_detections - 3etections - 32
{'M': 8, 'N': 40, 'D': 51, 'T': 1000, 'seed': 3, 'num_SM_events': 32, 'SM_total_spikes': 10, 'noise': 100}
2587 Windows
5 patterns found...s... 80% 10.01 | opt_cutoff - 0.61 | most_detections - 31tections - 31
{'M': 8, 'N': 40, 'D': 51, 'T': 1000, 'seed': 4, 'num_SM_events': 32, 'SM_total_spikes': 10, 'noise': 100}
2560 Windows
8 patterns found...s... 

In [17]:
df.groupby(["M"])

In [24]:
# Calculate the correlation matrix
correlation_matrix = df.corr()

# Display the correlation matrix
print(correlation_matrix)

                        M         N         D   T  num_SM_events  \
M                1.000000 -0.019511 -0.003807 NaN      -0.022546   
N               -0.019511  1.000000 -0.003488 NaN      -0.020657   
D               -0.003807 -0.003488  1.000000 NaN      -0.004031   
T                     NaN       NaN       NaN NaN            NaN   
num_SM_events   -0.022546 -0.020657 -0.004031 NaN       1.000000   
SM_total_spikes -0.024370 -0.022329 -0.004357 NaN      -0.025802   
noise           -0.031629 -0.028979 -0.005655 NaN      -0.033487   
window_time      0.331701 -0.062696  0.011927 NaN       0.448662   
cluster_time     0.404064  0.059379 -0.059149 NaN       0.476124   
sequence_time   -0.049144 -0.004694 -0.015947 NaN       0.010094   
total_time       0.118995 -0.006469  0.026695 NaN       0.410689   
total_patterns  -0.102824  0.030625 -0.003024 NaN      -0.030117   

                 SM_total_spikes     noise  window_time  cluster_time  \
M                      -0.024370 -0.031629

C:\Users\miles\AppData\Local\Temp\ipykernel_20648\2448198415.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix = df.corr()


In [ ]:
plt.figure